# Database programming

In [1]:
%pwd

'/Users/hayleeoyler/MEDS/spring/EDS-213/bren-eds213-data/week6'

In [2]:
%cd ../database/

/Users/hayleeoyler/MEDS/spring/EDS-213/bren-eds213-data/database


In [3]:
import duckdb

ModuleNotFoundError: No module named 'duckdb'